In [1]:
# !pip install git+https://github.com/rcmalli/keras-vggface.git
# !pip install keras_vggface
# !pip install keras_applications

In [2]:
from collections import defaultdict
from glob import glob
from random import choice, sample

import cv2
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, LayerNormalization, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2

from tf2_keras_vggface.utils import preprocess_input
from tf2_keras_vggface.vggface import VGGFace

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

2021-08-09 18:14:35.970063: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Using VGGFace compatible with TensorFlow2.


In [3]:
# Hyperparameters
BASE_MODEL = 'senet50'
INPUT_SHAPE = (224, 224,)
IGNORE_BOTTOM_NLAYERS_TUNE = 1
IGNORE_TOP_NLAYERS_TUNE = 1
FINE_TUNE = False

# Modify paths as per your method of saving them
BASE_PATH = "/root/KinshipRecognition"
TRAIN_FILE_PATH = f"{BASE_PATH}/data/aug_train_ds.csv"
TRAIN_FOLDERS_PATH = f"{BASE_PATH}/data/train/train-faces/"

# Output file
MODEL_NAME = "ensemble_vggface_senet50_notune_dense32-128-32_drop05"

# All images belonging to families F09** will be used to create the validation set while training the model
# For final submission, you can add these to the training data as well
# val_families_list = ["F06"]
val_families_list = ["F02","F04","F06","F08", "F09"]

In [4]:
def get_train_val(family_name):

    val_families = family_name

    all_images = glob(TRAIN_FOLDERS_PATH + "*/*/*.jpg")
    train_images = [x for x in all_images if val_families not in x]
    val_images = [x for x in all_images if val_families in x]

    train_person_to_images_map = defaultdict(list)

    ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

    for x in train_images:
        train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

    val_person_to_images_map = defaultdict(list)

    for x in val_images:
        val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)
    relationships = pd.read_csv(TRAIN_FILE_PATH)
    relationships = list(zip(relationships.p1.values, relationships.p2.values, relationships.relationship.values))
    relationships = [(x[0],x[1],x[2]) for x in relationships if x[0][:10] in ppl and x[1][:10] in ppl]    

    train = [x for x in relationships if val_families not in x[0]]
    val = [x for x in relationships if val_families in x[0]]
    return train, val, train_person_to_images_map, val_person_to_images_map

In [5]:
def read_img(path, input_shape):
    img = cv2.imread(path, -1)
    img = cv2.resize(img, input_shape)
    img = cv2.normalize(img,  np.zeros(img.shape[:2]), 0, 255, cv2.NORM_MINMAX)
    return np.array(img).astype(np.float)

In [6]:
def gen(list_tuples, person_to_images_map, input_shape, batch_size=16, normalization='base'):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size)
        
        # All the samples are taken from train_ds.csv, labels are in the labels column
        labels = []
        for tup in batch_tuples:
            labels.append(tup[2])
        labels = np.array(labels)

        # Original images preprocessed
        X1 = [x[0] for x in batch_tuples]
        X1 = np.array([read_img(TRAIN_FOLDERS_PATH + x, input_shape) for x in X1])
        
        X2 = [x[1] for x in batch_tuples]
        X2 = np.array([read_img(TRAIN_FOLDERS_PATH + x, input_shape) for x in X2])
        
        # Mirrored images
        X1_mirror = np.asarray([cv2.flip(x, 1) for x in X1])
        X2_mirror = np.asarray([cv2.flip(x, 1) for x in X2])
        X1 = np.r_[X1, X1_mirror]
        X2 = np.r_[X2, X2_mirror]
        
        yield [X1, X2], np.r_[labels, labels]

In [7]:
def baseline_model(model_name, fine_tune=True):
    input_1 = Input(shape=INPUT_SHAPE + (3,))
    input_2 = Input(shape=INPUT_SHAPE + (3,))

    backbone = VGGFace(model=model_name, include_top=False)
    for x in backbone.layers:
        x.trainable = False

    if fine_tune:
        for x in backbone.layers[:IGNORE_BOTTOM_NLAYERS_TUNE]:
            x.trainable = False
        if IGNORE_TOP_NLAYERS_TUNE == 0:
            for x in backbone.layers[IGNORE_BOTTOM_NLAYERS_TUNE:]:
                x.trainable = True
        else:
            for x in backbone.layers[IGNORE_BOTTOM_NLAYERS_TUNE:-IGNORE_TOP_NLAYERS_TUNE]:
                x.trainable = True

    for x in backbone.layers:
        print(x.name, x.trainable)

    x1 = backbone(input_1)
    x2 = backbone(input_2)

    x1 = GlobalAvgPool2D()(x1)
    x2 = GlobalAvgPool2D()(x2)

    x1 = LayerNormalization(axis=-1, epsilon=0.001, center=False, scale=False)(x1)
    x2 = LayerNormalization(axis=-1, epsilon=0.001, center=False, scale=False)(x2)

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])
    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    x4 = Subtract()([x1_, x2_])
    x5 = Multiply()([x1, x2])
    x = Concatenate(axis=-1)([x3, x4, x5])
        
#     x = LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=True)(x)
    x = Dense(32, activation="relu")(x)
    x = Dropout(0.05)(x)    
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.05)(x)    
    x = Dense(32, activation="tanh")(x)
#     x = LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=False)(x)
    x = Dropout(0.05)(x)    
    out = Dense(1, kernel_regularizer=L2(.01), activation="sigmoid")(x)

    model = Model([input_1, input_2], out)
    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))
    model.summary()
    
    return model

In [8]:
for i in range(len(val_families_list)):

    print('##############################')
    print(f'Iteration {i}: Validation on {val_families_list[i]}')
    print('##############################')

    train, val, train_person_to_images_map, val_person_to_images_map = get_train_val(val_families_list[i])
    file_path = f"{BASE_PATH}/log/model/{MODEL_NAME}_{i}.h5"
    checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.3, patience=30, verbose=1)
    callbacks_list = [checkpoint, reduce_on_plateau]
    
    model = baseline_model(BASE_MODEL, fine_tune=FINE_TUNE)
    
    history = model.fit(gen(train, train_person_to_images_map, INPUT_SHAPE, batch_size=16), 
                        validation_data=gen(val, val_person_to_images_map, INPUT_SHAPE, batch_size=16), 
                        epochs=25, steps_per_epoch=300, validation_steps=200,
                        verbose=1, callbacks=callbacks_list, 
                        use_multiprocessing=False, workers=1)

##############################
Iteration 0: Validation on F02
##############################


2021-08-09 18:14:56.924934: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-09 18:14:56.926006: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-09 18:14:56.984028: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-08-09 18:14:56.984091: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-08-09 18:14:56.987599: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-08-09 18:14:56.987709: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

104947712/104944616 [==============================] - 1s 0us/step
input_3 False
conv1/7x7_s2 False
conv1/7x7_s2/bn False
activation False
max_pooling2d False
conv2_1_1x1_reduce False
conv2_1_1x1_reduce/bn False
activation_1 False
conv2_1_3x3 False
conv2_1_3x3/bn False
activation_2 False
conv2_1_1x1_increase False
conv2_1_1x1_increase/bn False
global_average_pooling2d False
reshape False
conv2_1_1x1_down False
activation_3 False
conv2_1_1x1_up False
activation_4 False
conv2_1_1x1_proj False
multiply False
conv2_1_1x1_proj/bn False
add False
activation_5 False
conv2_2_1x1_reduce False
conv2_2_1x1_reduce/bn False
activation_6 False
conv2_2_3x3 False
conv2_2_3x3/bn False
activation_7 False
conv2_2_1x1_increase False
conv2_2_1x1_increase/bn False
global_average_pooling2d_1 False
reshape_1 False
conv2_2_1x1_down False
activation_8 False
conv2_2_1x1_up False
activation_9 False
multiply_1 False
add_1 False
activation_10 False
conv2_3_1x1_reduce False
conv2_3_1x1_reduce/bn False
activation_11 

2021-08-09 18:15:01.620275: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-09 18:15:01.639189: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2300000000 Hz


Epoch 1/25


...


Epoch 00011: val_acc did not improve from 0.77188
Epoch 12/25
300/300 [==============================] - 47s 156ms/step - loss: 0.3935 - acc: 0.8339 - val_loss: 0.5416 - val_acc: 0.7569

Epoch 00012: val_acc did not improve from 0.77188
Epoch 13/25
300/300 [==============================] - 47s 157ms/step - loss: 0.3731 - acc: 0.8464 - val_loss: 0.5592 - val_acc: 0.7572

Epoch 00013: val_acc did not improve from 0.77188
Epoch 14/25
300/300 [==============================] - 48s 159ms/step - loss: 0.3789 - acc: 0.8462 - val_loss: 0.5499 - val_acc: 0.7547

Epoch 00014: val_acc did not improve from 0.77188
Epoch 15/25
300/300 [==============================] - 47s 157ms/step - loss: 0.3493 - acc: 0.8592 - val_loss: 0.5593 - val_acc: 0.7614

Epoch 00015: val_acc did not improve from 0.77188
Epoch 16/25
300/300 [==============================] - 47s 158ms/step - loss: 0.3501 - acc: 0.8575 - val_loss: 0.5682 - val_acc: 0.7527

Epoch 00016: val_acc did not improve from 0.77188
Epoch 17/25
30

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_senet50 (Functional)    (None, None, None, 2 26092144    input_10[0][0]                   
                                                                 input_11[0][0]                   
__________________________________________________________________________________________________
global_average_pooling2d_70 (Gl (None, 2048)         0           vggface_senet50[0][0]      

Epoch 10/25
300/300 [==============================] - 47s 157ms/step - loss: 0.4354 - acc: 0.8114 - val_loss: 0.5275 - val_acc: 0.7509

Epoch 00010: val_acc did not improve from 0.75453
Epoch 11/25
300/300 [==============================] - 48s 159ms/step - loss: 0.4230 - acc: 0.8108 - val_loss: 0.5402 - val_acc: 0.7511

Epoch 00011: val_acc did not improve from 0.75453
Epoch 12/25
300/300 [==============================] - 48s 160ms/step - loss: 0.4168 - acc: 0.8232 - val_loss: 0.5271 - val_acc: 0.7544

Epoch 00012: val_acc did not improve from 0.75453
Epoch 13/25
300/300 [==============================] - 48s 160ms/step - loss: 0.3859 - acc: 0.8366 - val_loss: 0.5106 - val_acc: 0.7653

Epoch 00013: val_acc improved from 0.75453 to 0.76531, saving model to /root/KinshipRecognition/log/model/ensemble_vggface_senet50_notune_dense32-128-32_drop05_3.h5
Epoch 14/25
300/300 [==============================] - 46s 155ms/step - loss: 0.3949 - acc: 0.8357 - val_loss: 0.5154 - val_acc: 0.7667



Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_senet50 (Functional)    (None, None, None, 2 26092144    input_13[0][0]                   
                                                                 input_14[0][0]                   
__________________________________________________________________________________________________
global_average_pooling2d_88 (Gl (None, 2048)         0           vggface_senet50[0][0]      

Epoch 11/25
300/300 [==============================] - 46s 153ms/step - loss: 0.3942 - acc: 0.8300 - val_loss: 0.6724 - val_acc: 0.6716

Epoch 00011: val_acc did not improve from 0.67656
Epoch 12/25
300/300 [==============================] - 46s 153ms/step - loss: 0.3788 - acc: 0.8425 - val_loss: 0.6687 - val_acc: 0.6697

Epoch 00012: val_acc did not improve from 0.67656
Epoch 13/25
300/300 [==============================] - 46s 153ms/step - loss: 0.3664 - acc: 0.8454 - val_loss: 0.6767 - val_acc: 0.6775

Epoch 00013: val_acc improved from 0.67656 to 0.67750, saving model to /root/KinshipRecognition/log/model/ensemble_vggface_senet50_notune_dense32-128-32_drop05_4.h5
Epoch 14/25
300/300 [==============================] - 47s 155ms/step - loss: 0.3442 - acc: 0.8576 - val_loss: 0.6724 - val_acc: 0.6794

Epoch 00014: val_acc improved from 0.67750 to 0.67937, saving model to /root/KinshipRecognition/log/model/ensemble_vggface_senet50_notune_dense32-128-32_drop05_4.h5
Epoch 15/25
300/300 [=

In [9]:
test_path = f"{BASE_PATH}/data/test/"
submission = pd.read_csv(f'{BASE_PATH}/data/test_ds.csv')
preds_for_sub = np.zeros(submission.shape[0])
all_preds = list()
for i in range(len(val_families_list)):

    print('##############################')
    print(f'Iteration {i}: Validation on {val_families_list[i]}')
    print('##############################')
    
    model = baseline_model(BASE_MODEL, fine_tune=FINE_TUNE)
    file_path = f"{BASE_PATH}/log/model/{MODEL_NAME}_{i}.h5"
    model.load_weights(file_path)

    # Predictions
    predictions = []
    for j in range(0, len(submission.p1.values), 32):
        X1 = submission.p1.values[j:j+32]
        X1 = np.array([read_img(test_path + x, INPUT_SHAPE) for x in X1])

        X2 = submission.p2.values[j:j+32]
        X2 = np.array([read_img(test_path + x, INPUT_SHAPE) for x in X2])

        pred = model.predict([X1, X2]).ravel().tolist()
        predictions += pred    
    
    all_preds.append(np.array(predictions))
    preds_for_sub += np.array(predictions) / len(val_families_list)

    
all_preds = np.asarray(all_preds)
submission['score'] = preds_for_sub
pd.DataFrame(all_preds).to_csv(f"{BASE_PATH}/log/results/{MODEL_NAME}_allpreds.csv", index=False)
submission.to_csv(f"{BASE_PATH}/log/results/{MODEL_NAME}.csv", index=False)

##############################
Iteration 0: Validation on F02
##############################
input_18 False
conv1/7x7_s2 False
conv1/7x7_s2/bn False
activation_405 False
max_pooling2d_5 False
conv2_1_1x1_reduce False
conv2_1_1x1_reduce/bn False
activation_406 False
conv2_1_3x3 False
conv2_1_3x3/bn False
activation_407 False
conv2_1_1x1_increase False
conv2_1_1x1_increase/bn False
global_average_pooling2d_90 False
reshape_80 False
conv2_1_1x1_down False
activation_408 False
conv2_1_1x1_up False
activation_409 False
conv2_1_1x1_proj False
multiply_100 False
conv2_1_1x1_proj/bn False
add_80 False
activation_410 False
conv2_2_1x1_reduce False
conv2_2_1x1_reduce/bn False
activation_411 False
conv2_2_3x3 False
conv2_2_3x3/bn False
activation_412 False
conv2_2_1x1_increase False
conv2_2_1x1_increase/bn False
global_average_pooling2d_91 False
reshape_81 False
conv2_2_1x1_down False
activation_413 False
conv2_2_1x1_up False
activation_414 False
multiply_101 False
add_81 False
activation_415 Fal

##############################
Iteration 1: Validation on F04
##############################
input_21 False
conv1/7x7_s2 False
conv1/7x7_s2/bn False
activation_486 False
max_pooling2d_6 False
conv2_1_1x1_reduce False
conv2_1_1x1_reduce/bn False
activation_487 False
conv2_1_3x3 False
conv2_1_3x3/bn False
activation_488 False
conv2_1_1x1_increase False
conv2_1_1x1_increase/bn False
global_average_pooling2d_108 False
reshape_96 False
conv2_1_1x1_down False
activation_489 False
conv2_1_1x1_up False
activation_490 False
conv2_1_1x1_proj False
multiply_120 False
conv2_1_1x1_proj/bn False
add_96 False
activation_491 False
conv2_2_1x1_reduce False
conv2_2_1x1_reduce/bn False
activation_492 False
conv2_2_3x3 False
conv2_2_3x3/bn False
activation_493 False
conv2_2_1x1_increase False
conv2_2_1x1_increase/bn False
global_average_pooling2d_109 False
reshape_97 False
conv2_2_1x1_down False
activation_494 False
conv2_2_1x1_up False
activation_495 False
multiply_121 False
add_97 False
activation_496 F

##############################
Iteration 2: Validation on F06
##############################
input_24 False
conv1/7x7_s2 False
conv1/7x7_s2/bn False
activation_567 False
max_pooling2d_7 False
conv2_1_1x1_reduce False
conv2_1_1x1_reduce/bn False
activation_568 False
conv2_1_3x3 False
conv2_1_3x3/bn False
activation_569 False
conv2_1_1x1_increase False
conv2_1_1x1_increase/bn False
global_average_pooling2d_126 False
reshape_112 False
conv2_1_1x1_down False
activation_570 False
conv2_1_1x1_up False
activation_571 False
conv2_1_1x1_proj False
multiply_140 False
conv2_1_1x1_proj/bn False
add_112 False
activation_572 False
conv2_2_1x1_reduce False
conv2_2_1x1_reduce/bn False
activation_573 False
conv2_2_3x3 False
conv2_2_3x3/bn False
activation_574 False
conv2_2_1x1_increase False
conv2_2_1x1_increase/bn False
global_average_pooling2d_127 False
reshape_113 False
conv2_2_1x1_down False
activation_575 False
conv2_2_1x1_up False
activation_576 False
multiply_141 False
add_113 False
activation_5

##############################
Iteration 3: Validation on F08
##############################
input_27 False
conv1/7x7_s2 False
conv1/7x7_s2/bn False
activation_648 False
max_pooling2d_8 False
conv2_1_1x1_reduce False
conv2_1_1x1_reduce/bn False
activation_649 False
conv2_1_3x3 False
conv2_1_3x3/bn False
activation_650 False
conv2_1_1x1_increase False
conv2_1_1x1_increase/bn False
global_average_pooling2d_144 False
reshape_128 False
conv2_1_1x1_down False
activation_651 False
conv2_1_1x1_up False
activation_652 False
conv2_1_1x1_proj False
multiply_160 False
conv2_1_1x1_proj/bn False
add_128 False
activation_653 False
conv2_2_1x1_reduce False
conv2_2_1x1_reduce/bn False
activation_654 False
conv2_2_3x3 False
conv2_2_3x3/bn False
activation_655 False
conv2_2_1x1_increase False
conv2_2_1x1_increase/bn False
global_average_pooling2d_145 False
reshape_129 False
conv2_2_1x1_down False
activation_656 False
conv2_2_1x1_up False
activation_657 False
multiply_161 False
add_129 False
activation_6

##############################
Iteration 4: Validation on F09
##############################
input_30 False
conv1/7x7_s2 False
conv1/7x7_s2/bn False
activation_729 False
max_pooling2d_9 False
conv2_1_1x1_reduce False
conv2_1_1x1_reduce/bn False
activation_730 False
conv2_1_3x3 False
conv2_1_3x3/bn False
activation_731 False
conv2_1_1x1_increase False
conv2_1_1x1_increase/bn False
global_average_pooling2d_162 False
reshape_144 False
conv2_1_1x1_down False
activation_732 False
conv2_1_1x1_up False
activation_733 False
conv2_1_1x1_proj False
multiply_180 False
conv2_1_1x1_proj/bn False
add_144 False
activation_734 False
conv2_2_1x1_reduce False
conv2_2_1x1_reduce/bn False
activation_735 False
conv2_2_3x3 False
conv2_2_3x3/bn False
activation_736 False
conv2_2_1x1_increase False
conv2_2_1x1_increase/bn False
global_average_pooling2d_163 False
reshape_145 False
conv2_2_1x1_down False
activation_737 False
conv2_2_1x1_up False
activation_738 False
multiply_181 False
add_145 False
activation_7

In [10]:
print(np.sum(preds_for_sub <= 0.5))
print(len(preds_for_sub), '\n')
for line in preds_for_sub:
    print(line)

2102
3000 

0.1405569225549698
0.19679031372070313
0.16570896208286287
0.6744260787963867
0.12587559446692465
0.7159622609615326
0.26796026304364207
0.04645086880773306
0.6873768150806427
0.16564698293805122
0.05537792593240738
0.5209225058555603
0.07266951277852059
0.7899823665618897
0.07442988157272339
0.8907375574111939
0.3614255875349045
0.5143779695034028
0.5198260933160782
0.08709712624549866
0.7790850162506103
0.04848393853753806
0.4948931962251663
0.12991649545729161
0.5226746797561646
0.11144632752984762
0.11533686965703964
0.22458265423774718
0.07578394748270512
0.24775723218917844
0.3215015590190887
0.23117848113179207
0.5242587119340897
0.2194188579916954
0.30670291781425474
0.14785592854022978
0.16970993727445605
0.7758516192436218
0.4229318410158157
0.10538024418056012
0.0606456022709608
0.12241012752056121
0.3772853091359139
0.15519112423062326
0.5996911108493804
0.09204949587583541
0.3739614367485047
0.03901573587208986
0.5858844161033631
0.04598416462540627
0.165317433